In [1]:
import keras
from keras.models import Model,load_model
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score


Using TensorFlow backend.


In [2]:
model = load_model("cifar4_CNN_Epoch_100_Daug_True.h5")
# model.summary()

In [3]:
NUM_CLASSES = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

# Normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 
x_test /= 255

In [4]:
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'],
              optimizer='adam')

score = model.evaluate(x_test,y_test)
score

10000/10000 [==============================] - 4s 350us/step


[0.5269968831062317, 0.8828]

In [5]:
adv_x_train = np.load("adversarial_x_train.npy")
adv_y_train = np.load("adversarial_y_train.npy")
adv_x_test = np.load("adversarial_x_test.npy")
adv_y_test = np.load("adversarial_y_test.npy")

In [41]:
model_2 = Model(inputs=model.input, outputs=model.layers[24].output)
x_train_flat = model_2.predict(x_train)
y_train_flat = np.argmax(y_train,axis=1)
x_test_flat =  model_2.predict(x_test)
y_test_flat =  np.argmax(y_test,axis=1)
x_test_adv_flat = model_2.predict(adv_x_test)
y_test_adv_flat = np.argmax(adv_y_test,axis=1)

x_train_adv_flat = model_2.predict(adv_x_train)
y_train_adv_flat = np.argmax(adv_y_train,axis=1)

In [42]:
rng = 1000
clf = svm.SVC(kernel='linear',verbose=True)

In [43]:
clf.fit(x_train_flat[0:rng], y_train_flat[0:rng])

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=True)

In [44]:
y_pred = clf.predict(x_test_flat[0:rng])
accuracy_score(y_test_flat[0:rng], y_pred[0:rng])

0.822

In [45]:
y_pred_adv = clf.predict(x_test_adv_flat[0:rng])
accuracy_score(y_test_adv_flat[0:rng], y_pred_adv[0:rng])

0.416

In [46]:
clf2 = svm.LinearSVC(random_state=0, tol=1e-5)
clf2.fit(x_train_flat, y_train_flat)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=1e-05, verbose=0)

In [47]:
y_pred = clf2.predict(x_test_flat)
accuracy_score(y_test_flat, y_pred)

0.8753

In [48]:
y_pred_adv = clf2.predict(x_test_adv_flat)
accuracy_score(y_test_adv_flat, y_pred_adv)

0.4480016107923085

In [52]:
y_pred_adv_2 = clf2.predict(x_train_adv_flat)
accuracy_score(y_train_adv_flat, y_pred_adv_2)

0.5052402450822315